# Chapter5-6

## 勾配累積
小さなバッチサイズで計算した勾配を集約する<br>
$\rightarrow$メモリ使用量を減少しつつ実質のバッチサイズを増やすことができる

In [ ]:
# 勾配累積を使用しない場合の例
# これ単体では実装できない。あくまでこういう書き方でできるという例
from torch.utils.data import DataLoader

per_device_train_batch_size = 32 # バッチサイズ

# DataLoaderでミニバッチを作成する
data_loader = DataLoader(
    dataset, batch_size=per_device_train_batch_size
)

for batch_idx, inputs in enumerate(data_loader):
  loss = model.compute_loss(inputs) # 損失の計算
  loss.backward() # 誤差逆伝搬で勾配を計算、パラメータに保持させる
  optimizer.step() # パラメータ更新
  optimizer.zero_grad() # パラメータが保持している勾配を消去する

In [ ]:
# 勾配累積を使用する場合
# これ単体では実装できない。あくまでこういう書き方でできるという例
per_device_train_batch_size = 8 # 一回の勾配計算におけるバッチサイズ
gradient_accumulation_steps = 4 # 勾配計算の累積回数

data_loader = DataLoader(
    dataset, batch_size=per_device_train_batch_size
)

for batch_idx, inputs in enumerate(data_loader):
  loss = model.compute_loss(inputs) # 損失を計算

  # 事例毎の平均値とするため累積回数で損失を割る
  loss = loss / gradient_accumulation_steps

  # 誤差逆伝搬で勾配を計算、パラメータに保持
  # パラメータがすでに勾配を持っているとき、勾配は足し合わされる
  loss.backward()

  # 指定したステップの数だけ勾配が累積されたらパラメータ更新
  if (batch_idx + 1) % gradient_accumulation_steps == 0:
    optimizer.step() # パラメータ更新
    optimizer.zero_grad() # パラメータが保持している勾配を消去

パラメータの更新に用いられているバッチサイズは<br>
`per_device_train_batch_size`と`gradient_accumulation_steps`を乗算した値
